In [11]:
import pandas as pd
import numpy as np
import streamlit as st

In [12]:
cross_reference = pd.read_csv('Cross-Reference Sheet.csv')
winners_losers = pd.read_csv("Winners and Losers.csv")

In [13]:
player_decks = cross_reference.drop("Unnamed: 0", axis=1)
match_results = winners_losers.drop("Unnamed: 0", axis=1)

In [14]:
#Assisted by ChatGPT 4.0
# Merge to add winner's deck
match_results = match_results.merge(player_decks, left_on='Winners', right_on='Name', how='left')
match_results.rename(columns={'Deck': 'winner_deck'}, inplace=True)
match_results.drop('Name', axis=1, inplace=True)

# Merge to add loser's deck
match_results = match_results.merge(player_decks, left_on='Losers', right_on='Name', how='left')
match_results.rename(columns={'Deck': 'loser_deck'}, inplace=True)
match_results.drop('Name', axis=1, inplace=True)

match_results[match_results["winner_deck"] == "Arceus"]

,Winners,Losers,winner_deck,loser_deck
14,landen kaetler,dean nezam,Arceus,Lost Zone Box
55,travis beckwith,brian peavy,Arceus,Roaring Moon
56,makani tran,brandon flowers,Arceus,Gardevoir
64,landen kaetler,chris brewer,Arceus,Lost Box Paradox
101,jacob walton,connor woolf,Arceus,Miraidon
109,landen kaetler,alex krekeler,Arceus,Inteleon Urshifu
142,jacob walton,omar taha,Arceus,Gardevoir
202,makani tran,amy jones,Arceus,Charizard Arceus
238,makani tran,brendan cevasco,Arceus,Miraidon
286,makani tran,alec ward,Arceus,Miraidon Flaaffy


In [15]:
#Assisted by ChatGPT 4.0
# Fill NaN values with a placeholder string
match_results['winner_deck'] = match_results['winner_deck'].fillna('Unknown')
match_results['loser_deck'] = match_results['loser_deck'].fillna('Unknown')

# Create a new column for sorted matchups
match_results['matchup'] = match_results.apply(lambda row: tuple(sorted([row['winner_deck'], row['loser_deck']])), axis=1)

matchup_stats = match_results.groupby('matchup').apply(
    lambda group: pd.Series({
        'total_matches': len(group),
        'wins': sum(group['winner_deck'] == group['matchup'].apply(lambda x: x[0]))
    })
).reset_index()

# Calculate win rate
matchup_stats['win_rate'] = matchup_stats['wins'] / matchup_stats['total_matches']

print(matchup_stats)

                             matchup  total_matches  wins  win_rate
0                (Arceus, Charizard)              2     0      0.00
1         (Arceus, Charizard Arceus)              1     1      1.00
2                (Arceus, Gardevoir)              2     2      1.00
3                (Arceus, Gholdengo)              1     0      0.00
4         (Arceus, Inteleon Urshifu)              1     1      1.00
..                               ...            ...   ...       ...
104     (Roaring Moon, Roaring Moon)             17    17      1.00
105  (Roaring Moon, Snorlax Pidgeot)              1     1      1.00
106    (Roaring Moon, Snorlax Stall)              4     1      0.25
107          (Roaring Moon, Unknown)              4     1      0.25
108   (Snorlax Stall, Snorlax Stall)              1     1      1.00

[109 rows x 4 columns]


C:\Users\noahm\AppData\Local\Temp\ipykernel_145948\1290745155.py:9: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [16]:
#Assisted by ChatGPT 4.0
#import matplotlib.pyplot as plt
import plotly.express as px

# Extract unique decks
decks = set()
for matchup in matchup_stats['matchup']:
    decks.update(matchup)
decks = sorted(decks)

# Create a matrix for the heatmap
heatmap_data = pd.DataFrame(index=decks, columns=decks, data=0.0)
for index, row in matchup_stats.iterrows():
    deck1, deck2 = row['matchup']
    win_rate = row['wins'] / row['total_matches']
    heatmap_data.loc[deck1, deck2] = win_rate
    heatmap_data.loc[deck2, deck1] = 1 - win_rate  # Assuming symmetrical matchups

# Adjust for mirror matchups
for deck in decks:
    heatmap_data.loc[deck, deck] = 0.5

# Drop the 'Unknown' row and column
heatmap_data = heatmap_data.drop('Unknown', axis=0)  # Drop row
heatmap_data = heatmap_data.drop('Unknown', axis=1)  # Drop column

# Convert win rates to percentages
heatmap_data_percent = heatmap_data * 100

# Format the annotations as percentages
annotations = heatmap_data_percent.applymap(lambda x: f'{x:0.1f}%')

import plotly.express as px

# Create the heatmap with Plotly
fig = px.imshow(heatmap_data_percent,
                labels=dict(x='Deck 1', y='Deck 2', color='Win Rate (%)'),
                x=heatmap_data_percent.columns,
                y=heatmap_data_percent.index,
                text_auto=True,  # Automatically add text
                aspect='auto',  # Adjust the aspect ratio
                color_continuous_scale=['red', 'yellow', 'green'])  # Red to Blue color scale, reversed

fig.update_layout(title='Win Rates for Pokemon TCG Matchups',
                  xaxis_title='Deck 1',
                  yaxis_title='Deck 2')

st.plotly_chart(fig)

C:\Users\noahm\AppData\Local\Temp\ipykernel_145948\1702325503.py:31: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'Deck 1: %{x}<br>Deck 2: %{y}<br>Win Rate (%): %{z}<extra></extra>',
              'name': '0',
              'texttemplate': '%{z}',
              'type': 'heatmap',
              'x': array(['Arceus', 'Charizard', 'Charizard Arceus', 'Chien-Pao Baxcalibur',
                          'Entei Iron Valiant', 'Gardevoir', 'Gholdengo', 'Giratina LZ Box',
                          'Inteleon Urshifu', 'Klawf Electrode', 'Lost Box Charizard',
                          'Lost Box Paradox', 'Lost Zone Box', 'Lugia Archeops', 'Mew Genesect',
                          'Miraidon', 'Miraidon Flaaffy', 'Roaring Moon', 'Snorlax Pidgeot',
                          'Snorlax Stall'], dtype=object),
              'xaxis': 'x',
              'y': array(['Arceus', 'Charizard', 'Charizard Arceus', 'Chien-Pao Baxcalibur',
                          'Entei Iron Valiant', 'Gardevoir', 'Gholdengo', 'Giratina LZ Box',
                          'Inteleon Urshifu', 'Klawf Electrode', 'Lost Box Charizard',
                          'Lost Box Paradox', 'Lost Zone Box', 'Lugia Archeops', 'Mew Genesect',
                          'Miraidon', 'Miraidon Flaaffy', 'Roaring Moon', 'Snorlax Pidgeot',
                          'Snorlax Stall'], dtype=object),
              'yaxis': 'y',
              'z': array([[ 50.        ,   0.        , 100.        , ...,  66.66666667,
                             0.        ,   0.        ],
                          [100.        ,  50.        , 100.        , ...,  50.        ,
                             0.        ,   0.        ],
                          [  0.        ,   0.        ,  50.        , ...,   0.        ,
                             0.        ,   0.        ],
                          ...,
                          [ 33.33333333,  50.        , 100.        , ...,  50.        ,
                           100.        ,  25.        ],
                          [  0.        ,   0.        ,   0.        , ...,   0.        ,
                            50.        ,   0.        ],
                          [  0.        , 100.        ,   0.        , ...,  75.        ,
                             0.        ,  50.        ]])}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Win Rate (%)'}},
                             'colorscale': [[0.0, 'red'], [0.5, 'yellow'], [1.0,
                                            'green']]},
               'margin': {'t': 60},
               'template': '...',
               'title': {'text': 'Win Rates for Pokemon TCG Matchups'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Deck 1'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'Deck 2'}}}
})